In [ ]:
# Installments of required tables
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#  BigQuery API activation
from google.colab import auth
auth.authenticate_user()

In [ ]:
# big query add-on installation

!pip install --quiet google-cloud-bigquery
from google.cloud import bigquery

In [ ]:
# BigQuery client initiation
client = bigquery.Client(project="homecredit-478707")

# From BigQuery
query = """
SELECT *
FROM `homecredit-478707.Homecredit_Tables.Installments_payments`
"""
installments_payments = client.query(query).to_dataframe()

In [ ]:
installments_payments.info()

In [ ]:
installments_payments.head(3)

Why Installment Payments Cannot Use a Simple Primary Key

The installments_payments table contains one row per installment payment, but the dataset often includes multiple records for the same installment, caused by:

updates in the payment date

version changes in the repayment schedule

system-level duplicate entries

multiple recorded corrections or adjustments

In [ ]:
duplicates = installments_payments.duplicated(
    subset=["SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER"]
).sum()

print("Duplicate PK rows:", duplicates)
print("Total rows:", installments_payments.shape[0])


In [ ]:
duplicates_v2 = installments_payments.duplicated(
    subset=["SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER", "NUM_INSTALMENT_VERSION"]
).sum()

print("Duplicate PK rows with version:", duplicates_v2)


In [ ]:
# Keep only the latest payment entry per installment
installments_payments_sorted = installments_payments.sort_values(
    by=["SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER", "NUM_INSTALMENT_VERSION", "DAYS_ENTRY_PAYMENT"],
    ascending=[True, True, True, True, True]
)

installments_unique = installments_payments_sorted.drop_duplicates(
    subset=["SK_ID_CURR", "SK_ID_PREV", "NUM_INSTALMENT_NUMBER"],
    keep="last"
)

print("Original rows:", installments_payments.shape[0])
print("After deduplication:", installments_unique.shape[0])


How We Cleaned the Duplicates (Industry-Standard Approach)

To ensure a reliable and consistent installment-level dataset, we applied:

✔️ “Latest payment record” rule

For each installment, we kept the row with the most recent DAYS_ENTRY_PAYMENT, assuming it represents the most accurate and updated payment information.

This method:

eliminates duplicate payment logs

retains the latest corrected installment info

ensures a clean, one-row-per-installment structure

prepares the table for accurate behavior-based feature engineering

⭐ 2) Missing Value & Distribution — Do We Need Full EDA?
✔️ Short answer: No — not the classic EDA, and işte nedeni 👇

Installments table is transactional. Meaning:

It is not customer-level data

It is not application-level data

It is a time series of repayment actions

Therefore, missing values or distributions in the raw table do not have the same meaning as in application_train or previous_application.

⭐ Why missing values are NOT handled traditionally here
• DAYS_ENTRY_PAYMENT can be missing if the payment has not been made yet.

This is a behavioral signal, not a “missing value problem”.

• AMT_PAYMENT may be zero if the customer didn’t pay.

This is a payment default behavior, not something to impute.

• DAYS_INSTALMENT is always populated and not meant to be cleaned.
• NUM_INSTALMENT_VERSION may be duplicated for system reasons.

We already resolved this through deduplication.

Therefore:

❌ No median imputation
❌ No mode imputation
❌ No manual filling
❌ No column dropping

Because that would destroy behavioral patterns that models depend on.

⭐ 3) Why Distribution Plots Are Not Useful Here

For application_train:

histograms are meaningful

boxplots show real outliers

feature scaling matters

But for installments:

Installment values represent repeating monthly events.

Plotting distributions tells us nothing actionable because:

Each customer has a different loan structure

Every loan has 12–96+ installments

DAYS_INSTALMENT repeats in structured cycles

Amounts depend entirely on loan type, tenure, and schedule

📌 Distribution EDA is not only unhelpful, it’s actually misleading.

⭐ 4) What Matters Instead?

Installments tablosunda 1 numaralı amaç:

Extract behavioral signals

such as:

late payment count

early payment count

mean delay

overpayment ratio

underpayment frequency

max days late

payment discipline patterns

These FE features are far more meaningful than raw distributions.

Installments Table – Rationale Behind the Cleaning Approach

The installments table contains multiple recorded versions of the same installment due to payment updates and system-level replication.

A standard primary key does not exist; even the extended key including version fields is not unique.

To ensure consistent installment-level data, we kept the latest payment entry per installment, based on the maximum DAYS_ENTRY_PAYMENT value.

Missing values in this table reflect payment behavior (e.g., missed installments), not data quality problems, and therefore must not be imputed.

Distribution-based EDA is not meaningful for transactional time-series repayment data.

Instead, we engineered key behavioral features such as payment delay, underpayment and overpayment flags, payment ratios, and delinquency patterns.

These aggregated behavior-level signals significantly enhance the predictive power of the credit scoring model.

# Late / Early Payment FE

In [ ]:
# Use deduplicated version
installments = installments_unique.copy()

# -----------------------------------------------
# 1) Days past due (late payment)
# PAST_DUE = DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT
# -----------------------------------------------
installments["PAYMENT_DELAY"] = installments["DAYS_ENTRY_PAYMENT"] - installments["DAYS_INSTALMENT"]

# Late payment (positive delay)
installments["LATE_PAYMENT_FLAG"] = (installments["PAYMENT_DELAY"] > 0).astype(int)

# Early payment (negative delay)
installments["EARLY_PAYMENT_FLAG"] = (installments["PAYMENT_DELAY"] < 0).astype(int)

# On-time payment
installments["ON_TIME_PAYMENT_FLAG"] = (installments["PAYMENT_DELAY"] == 0).astype(int)


# ------------------------------------------------
# 2) Payment ratio
# PAYMENT_RATIO = AMT_PAYMENT / AMT_INSTALMENT
# ------------------------------------------------
installments["PAYMENT_RATIO"] = installments["AMT_PAYMENT"] / (installments["AMT_INSTALMENT"] + 1e-6)

# Underpayment (<1)
installments["UNDERPAYMENT_FLAG"] = (installments["PAYMENT_RATIO"] < 1).astype(int)

# Exact payment (=1)
installments["EXACT_PAYMENT_FLAG"] = (installments["PAYMENT_RATIO"].round(3) == 1).astype(int)

# Overpayment (>1)
installments["OVERPAYMENT_FLAG"] = (installments["PAYMENT_RATIO"] > 1).astype(int)


# ------------------------------------------------
# 3) Absolute delay magnitudes
# ------------------------------------------------
installments["LATE_DAYS"] = installments["PAYMENT_DELAY"].clip(lower=0)
installments["EARLY_DAYS"] = (-installments["PAYMENT_DELAY"]).clip(lower=0)


# Customer-Level Aggregation (SK_ID_CURR)

In [ ]:
# Aggregation dictionary for installments
agg_dict = {
    "PAYMENT_DELAY": ["mean", "max", "min", "std", "median"],
    "LATE_DAYS": ["mean", "max"],
    "EARLY_DAYS": ["mean", "max"],
    "LATE_PAYMENT_FLAG": ["mean", "sum"],
    "EARLY_PAYMENT_FLAG": ["mean", "sum"],
    "ON_TIME_PAYMENT_FLAG": ["mean", "sum"],
    "PAYMENT_RATIO": ["mean", "min", "max", "std"],
    "UNDERPAYMENT_FLAG": ["mean", "sum"],
    "EXACT_PAYMENT_FLAG": ["mean", "sum"],
    "OVERPAYMENT_FLAG": ["mean", "sum"],
    "AMT_INSTALMENT": ["mean", "sum", "max"],
    "AMT_PAYMENT": ["mean", "sum", "max"]
}

installments_agg = installments.groupby("SK_ID_CURR").agg(agg_dict)

# flatten multi-index
installments_agg.columns = [
    f"INSTAL_{col[0]}_{col[1].upper()}" for col in installments_agg.columns
]

installments_agg.reset_index(inplace=True)

print("Final installments_agg shape:", installments_agg.shape)
installments_agg.head()


Installments Aggregate – English Explanation
What the aggregated table represents

The installments_agg table contains customer-level repayment behavior features derived from over 12.8 million installment payment records. Each row corresponds to one customer (SK_ID_CURR), and each column summarizes a specific type of credit behavior such as delay, early payment, underpayment, or overpayment.

* Why these features matter

Installment payment behavior is one of the strongest predictors of credit default.
The following patterns highly correlate with repayment risk:

Late payments → indicate financial stress

Early payments → indicate financial discipline or low risk

Underpayments → cash flow problems

Overpayments → high repayment capacity

Payment ratio variability → financial instability

Large maximum late days → high delinquency tendency

The 32 engineered features capture these behaviors at scale.

* How the aggregation works (in simple terms)

We transformed raw installment-level records into interpretable customer-level summaries:

* Delay-based features

INSTAL_PAYMENT_DELAY_MEAN

INSTAL_PAYMENT_DELAY_MAX

INSTAL_LATE_DAYS_MAX

These quantify how late or early customers tend to pay their installments.

* Payment precision features

INSTAL_UNDERPAYMENT_FLAG_MEAN

INSTAL_EXACT_PAYMENT_FLAG_MEAN

INSTAL_OVERPAYMENT_FLAG_MEAN

These indicate how often customers paid less, exactly, or more than required.

* Payment ratio & amount patterns

INSTAL_PAYMENT_RATIO_MEAN

INSTAL_PAYMENT_RATIO_STD

INSTAL_AMT_PAYMENT_MAX

INSTAL_AMT_INSTALMENT_SUM

* These measure the payment discipline and loan burden capacity.

*Final structure
Final installments_agg shape: (339,587 customers, 32 features)


This means:

We successfully compressed 12.8 million transactional records

Into 32 highly predictive features

At customer level, ready for merging into application_train

Why these FE features improve model performance

Because they encode actual historical payment behavior, which is more predictive than demographic or application-level data.

Models like LightGBM benefit massively from:

payment delay patterns

delinquency frequency

repayment consistency

stability of payment amounts



In [ ]:
installments_agg.to_csv("installments_agg.csv", index = True)

In [ ]:
# Merge

# application_train = application_train.merge(installments_agg, on="SK_ID_CURR", how="left")
# print("After merging installments:", application_train.shape)
